<a href="https://colab.research.google.com/github/rajankumar-spec/CNN/blob/main/EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

Q. How many layers ?

Ans - if we use just convulution layer with kernal size of 3 X 3, & stride of 1, then no. of layers = size of the image /2 = 28/2 = 14

We will use maxpooling to
1. Decrease the no. of layers by increasing the receptive field (Rf depends on the stride used in last layer.)
2. it will reduce the size of image by half( max pooled by 2X2 & stride



We also use convolution layer with kernal size of 1X1 to combine the extracted features to generate features with high level patterns. it can increase/decrease no. of channel. kernel of 3x3 is used only for feature extraction & not mixing.


In [2]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -  OUtput? RF
#         self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
#         self.pool1 = nn.MaxPool2d(2, 2)
#         self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
#         self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
#         self.pool2 = nn.MaxPool2d(2, 2)
#         self.conv5 = nn.Conv2d(256, 512, 3)
#         self.conv6 = nn.Conv2d(512, 1024, 3)
#         self.conv7 = nn.Conv2d(1024, 10, 3)

#     def forward(self, x):
#         x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
#         x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
#         x = F.relu(self.conv6(F.relu(self.conv5(x))))
#         x = F.relu(self.conv7(x))
#         x = x.view(-1, 10)
#         return F.log_softmax(x)

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1) #input -  OUtput? RF
        self.conv2 = nn.Conv2d(8, 8, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.back1 = nn.BatchNorm2d(num_features=8)
        self.conv3 = nn.Conv2d(8, 16, 3, padding=1)
        self.conv4 = nn.Conv2d(16, 16, 3)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.back2 = nn.BatchNorm2d(num_features=16)
        self.dropout = nn.Dropout(0.1)
        self.conv5 = nn.Conv2d(16, 32, 3)
        self.conv6 = nn.Conv2d(32, 32, 3)
        self.back3 = nn.BatchNorm2d(num_features=32)
        self.fc1 = nn.Linear(128, 10)
        self.fc2 = nn.Linear(20, 10)

    def forward(self, x):

        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.back1(x)
        x = self.dropout(x)
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = self.back2(x)
        x = self.dropout(x)
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        # x = self.dropout(x)
        x = x.view(-1, 128)
        x = self.fc1(x)
        # x = self.fc2(x)
        return F.log_softmax(x)

In [4]:
# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
            Conv2d-2            [-1, 8, 28, 28]             584
         MaxPool2d-3            [-1, 8, 14, 14]               0
       BatchNorm2d-4            [-1, 8, 14, 14]              16
           Dropout-5            [-1, 8, 14, 14]               0
            Conv2d-6           [-1, 16, 14, 14]           1,168
            Conv2d-7           [-1, 16, 12, 12]           2,320
         MaxPool2d-8             [-1, 16, 6, 6]               0
       BatchNorm2d-9             [-1, 16, 6, 6]              32
          Dropout-10             [-1, 16, 6, 6]               0
           Conv2d-11             [-1, 32, 4, 4]           4,640
           Conv2d-12             [-1, 32, 2, 2]           9,248
           Linear-13                   [-1, 10]           1,290
Total params: 19,378
Trainable params: 

/tmp/ipython-input-3263285593.py:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [5]:
print(device)

cuda


In [6]:


torch.manual_seed(42)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
    transforms.RandomApply([transforms.CenterCrop(22), ], p=0.1),
    transforms.Resize((28, 28)),
    transforms.RandomRotation((-15., 15.), fill=0),
    transforms.ToTensor(),
    transforms.Normalize((0.2307,), (0.4081,)),
    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.2307,), (0.4081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


100%|██████████| 9.91M/9.91M [00:01<00:00, 5.04MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 134kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.27MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.22MB/s]


In [7]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()}, Accuracy: {correct}/{len(train_loader.dataset)} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.015, momentum=0.9)

for epoch in range(1,10):
    print("Epoch: ", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Epoch:  1


  0%|          | 0/469 [00:00<?, ?it/s]/tmp/ipython-input-3263285593.py:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.0634193941950798, Accuracy: 54595/60000 batch_id=468: 100%|██████████| 469/469 [00:24<00:00, 19.48it/s]



Test set: Average loss: 0.0559, Accuracy: 9821/10000 (98.21%)

Epoch:  2


loss=0.1045989990234375, Accuracy: 58460/60000 batch_id=468: 100%|██████████| 469/469 [00:23<00:00, 20.08it/s]



Test set: Average loss: 0.0336, Accuracy: 9895/10000 (98.95%)

Epoch:  3


...


Test set: Average loss: 0.0250, Accuracy: 9919/10000 (99.19%)

Epoch:  8


loss=0.06061778590083122, Accuracy: 59231/60000 batch_id=468: 100%|██████████| 469/469 [00:23<00:00, 19.80it/s]



Test set: Average loss: 0.0195, Accuracy: 9940/10000 (99.40%)

Epoch:  9


loss=0.040781375020742416, Accuracy: 59262/60000 batch_id=468: 100%|██████████| 469/469 [00:23<00:00, 20.09it/s]



Test set: Average loss: 0.0187, Accuracy: 9944/10000 (99.44%)

